In [26]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq, EarlyStoppingCallback
import evaluate
import nltk

# Download necessary NLTK resources
nltk.download('wordnet')  # Required for METEOR metric
nltk.download('punkt')    # Required for tokenization

# Set a random seed for reproducibility
np.random.seed(42)

# --- 1) Dataset Loading & Cleaning ---
print("\n### Loading dataset... ###")
dataset = load_dataset("opus100", "en-fr")

# Limit dataset size for faster training
train_dataset = dataset["train"].select(range(50000))  # 50k samples for training
val_dataset = dataset["validation"].select(range(2000))  # 2k for validation

# Function to remove duplicates and null values
def remove_duplicates_and_nulls(dataset):
    """Removes duplicate and null translations."""
    df = pd.DataFrame(dataset["translation"])
    df.dropna(inplace=True)  # Remove null values
    df.drop_duplicates(subset=["en", "fr"], inplace=True)  # Remove duplicates
    return dataset.select([i for i in range(len(df))])

train_dataset = remove_duplicates_and_nulls(train_dataset)
val_dataset = remove_duplicates_and_nulls(val_dataset)

# Function to remove empty or excessively long translations
def clean_dataset(example):
    """Removes empty translations and those exceeding 100 words."""
    en_text = example["translation"]["en"].strip()
    fr_text = example["translation"]["fr"].strip()
    return (0 < len(en_text) < 100) and (0 < len(fr_text) < 100)

train_dataset = train_dataset.filter(clean_dataset)
val_dataset = val_dataset.filter(clean_dataset)

print(f"\nDataset size after cleaning:\n- Train: {len(train_dataset)}\n- Validation: {len(val_dataset)}")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



### Loading dataset... ###


Filter:   0%|          | 0/49316 [00:01<?, ? examples/s]

Filter:   0%|          | 0/1997 [00:00<?, ? examples/s]


Dataset size after cleaning:
- Train: 32198
- Validation: 1109


In [27]:
# --- 2) Tokenization & Preprocessing ---
model_name = "Helsinki-NLP/opus-mt-en-fr"  # Selecting the best model
print(f"\nSelected Model: {model_name}")

# Tokenization setup
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    """Tokenizes input and output sentences."""
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)


Selected Model: Helsinki-NLP/opus-mt-en-fr


Map:   0%|          | 0/32198 [00:00<?, ? examples/s]

Map:   0%|          | 0/1109 [00:00<?, ? examples/s]

In [11]:
# --- 3) Model Training ---

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    predict_with_generate=True,
    logging_dir='./logs',
    fp16=True,
    warmup_steps=500,
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early stopping pour éviter surentraînement
)

print("\n### Entraînement du modèle en cours... ###")
trainer.train()




C:\Users\LENOVO\anaconda3\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-0a23f36f5018>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



### Entraînement du modèle en cours... ###


Epoch,Training Loss,Validation Loss
0,0.231600,0.240880
2,0.101100,0.213411


C:\Users\LENOVO\anaconda3\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=3018, training_loss=0.20725870673588526, metrics={'train_runtime': 62298.946, 'train_samples_per_second': 1.55, 'train_steps_per_second': 0.048, 'total_flos': 3273090523987968.0, 'train_loss': 0.20725870673588526, 'epoch': 2.9985096870342773})

In [13]:

# Sauvegarde du modèle en local
model.save_pretrained("./NLP_Project")
tokenizer.save_pretrained("./NLP_Project")
print(" Modèle entraîné et sauvegardé sous './NLP_Project' ###")


 Modèle entraîné et sauvegardé sous './NLP_Project' ###


In [19]:
# --- 4) Évaluation du Modèle ---
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
# Suppression de METEOR depuis evaluate, calcul manuel
from nltk.translate.meteor_score import meteor_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    bleu_score = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_scores = [meteor_score([ref.split()], pred.split()) for pred, ref in zip(decoded_preds, decoded_labels)]
    meteor_score_avg = sum(meteor_scores) / len(meteor_scores)
    
    return {
        "BLEU": bleu_score["score"],
        "ROUGE-L": rouge_score["rougeL"],
        "METEOR": meteor_score_avg
    }

print("\n### Évaluation en cours... ###")
predictions = trainer.predict(tokenized_val)
eval_results = compute_metrics((predictions.predictions, predictions.label_ids))
print("\nRésultats d'évaluation:", eval_results)



### Évaluation en cours... ###


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr


Résultats d'évaluation: {'BLEU': 28.440213961204403, 'ROUGE-L': 0.49509053222806454, 'METEOR': 0.4125180488897784}


In [25]:
# --- 5) Test sur des phrases significatives ---
def test_model(input_text):
    """Test du modèle sur une phrase donnée."""
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

test_sentences = [
    "I am A SPOC Student and this is my Natural language processing project",
    "That was not an easy project and hope i m gonna succeed",
    "Hello, how are you?",
    "The weather is nice today.",
    "I love machine learning!",
    "I am a SPOC Student and this is my ""NLP"" project"
]
print("\n### Tests de traduction ###")
for sentence in test_sentences:
    print(f"EN: {sentence} --> FR: {test_model(sentence)}")



### Tests de traduction ###
EN: I am A SPOC Student and this is my Natural language processing project --> FR: Je suis un étudiant SPOC et voici mon projet de traitement de la langue naturelle
EN: That was not an easy project and hope i m gonna succeed --> FR: Ce n'était pas un projet facile et j'espérais réussir
EN: Hello, how are you? --> FR: Comment allez-vous ?
EN: The weather is nice today. --> FR: Le temps est beaujourd'hui.
EN: I love machine learning! --> FR: J'adore l'apprentissage !
EN: I am a SPOC Student and this is my NLP project --> FR: Je suis étudiant SPOC et voici mon projet de PNG.
